In [3]:
import numpy as np
import pywt
simple_signal = np.array([1, 2, 3, 5, 12, 9, 8])
coeffs_pywt = pywt.wavedec(simple_signal, 'db8', level=3)

In [15]:
import pywt

# 定义要查看的小波
wavelet = pywt.Wavelet('bior5.5')
wavelet.rec_hi

[0.013456709459118716,
 -0.002694966880111507,
 -0.13670658466432914,
 -0.09350469740093886,
 0.47680326579848425,
 0.8995061097486484,
 0.47680326579848425,
 -0.09350469740093886,
 -0.13670658466432914,
 -0.002694966880111507,
 0.013456709459118716,
 0.0]

In [6]:
import numpy as np

# 上采样：在元素之间插入零
def upsample(signal, factor=2):
    upsampled = np.zeros(factor * len(signal))
    upsampled[::factor] = signal  # 将原始信号放在每隔factor位置
    return upsampled

# 卷积操作
def convolve(signal, filter):
    # 在卷积时使用 'same' 模式，确保长度一致
    return np.convolve(signal, filter, mode='same')

# IDWT实现
def idwt(approx_coeffs, detail_coeffs, wavelet_filters):
    # 从低频重构滤波器和高频重构滤波器中获取重构滤波器
    low_pass_reconstruction, high_pass_reconstruction = wavelet_filters
    
    current_signal = approx_coeffs  # 初始化为近似系数

    # 从最高层开始逐层重构
    for detail in detail_coeffs:
        # 对近似系数和细节系数进行上采样
        upsampled_approx = upsample(current_signal)
        upsampled_detail = upsample(detail)

        # 分别与低通滤波器和高通滤波器进行卷积
        recon_approx = convolve(upsampled_approx, low_pass_reconstruction)
        recon_detail = convolve(upsampled_detail, high_pass_reconstruction)

        # 将近似和细节部分相加，得到新的重构信号
        current_signal = recon_approx[:len(recon_detail)] + recon_detail[:len(recon_approx)]

    return current_signal

# 使用小波滤波器 'db8' 的重构滤波器
# 假设你已经知道了 db8 的重构滤波器
low_pass_reconstruction = [
    0.0033357252854737716, -0.012580751999015526, -0.006241490213011705, 0.07757149384006515, 
    -0.03224486958502952, -0.24229488706619015, 0.13842814590110342, 0.7243085284385744, 
    0.6038292697974729, 0.160102397974125, -0.09351809091712748, -0.065727528333722, 
    0.02752286553001629, 0.011031867509191737, -0.006262167954438661, -0.0012599933174382161
]

high_pass_reconstruction = [
    -0.0012599933174382161, -0.006262167954438661, 0.011031867509191737, 0.02752286553001629, 
    -0.065727528333722, -0.09351809091712748, 0.160102397974125, 0.6038292697974729, 
    0.7243085284385744, 0.13842814590110342, -0.24229488706619015, -0.03224486958502952, 
    0.07757149384006515, -0.006241490213011705, -0.012580751999015526, 0.0033357252854737716
]

# 使用 'db8' 的 DWT 分解结果（假设已经有 DWT 结果）
coeffs_pywt = [
    np.array([15.05807334]),  # Approximation coefficients (Level 3)
    np.array([-0.54066397]),  # Detail coefficients (Level 3)
    np.array([2.70554282, -1.27790182]),  # Detail coefficients (Level 2)
    np.array([-0.25767486, 0.75398363, -0.29917162, -0.06567155])  # Detail coefficients (Level 1)
]

# IDWT重构
reconstructed_signal_manual = idwt(coeffs_pywt[0], coeffs_pywt[1:], (low_pass_reconstruction, high_pass_reconstruction))

reconstructed_signal_manual


array([ 0.02212663,  0.053402  ,  0.0374743 , -0.05061264, -0.14780361,
       -0.02930767,  0.37198309,  0.21545059, -0.61658983, -0.79137161,
       -0.08033511,  0.78634302,  1.32088765,  1.69254534,  1.92077462,
        1.47096808])

In [5]:
import numpy as np
import pywt

# 定义简单信号
simple_signal = np.array([1, 2, 3, 5, 12, 9, 8])

# 使用pywt进行DWT分解
coeffs_pywt = pywt.wavedec(simple_signal, 'db8', level=3)

# 使用pywt进行IDWT重构
reconstructed_signal_pywt = pywt.waverec(coeffs_pywt, 'db8')

reconstructed_signal_pywt

array([ 1.,  2.,  3.,  5., 12.,  9.,  8.,  8.])